In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# want test code to take SparkSession as input because we want this to be able to run on a spark cluster
spark = SparkSession.builder.getOrCreate()

In [ ]:
input_prefix = 'input'